In [ ]:
import fiona
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt, MultiPolygon, Point
from geoalchemy2 import Geometry, WKTElement
import requests
from io import BytesIO
from os import path
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
from sklearn.decomposition import PCA
from scipy.stats import zscore, ttest_ind, spearmanr
import matplotlib.pyplot as plt
import folium
from folium.features import GeoJsonTooltip

In [ ]:
credentials = "../references/credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db, conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
srid = 4326
type_dict = {MultiPolygon: 'MultiPolygon', Point: 'Point'}

def create_wkt_element(geom, Type, srid=4326):
    type_str = type_dict[Type]
    if geom.geom_type != type_str:
        geom = Type([geom])
    return WKTElement(geom.wkt, srid)

In [ ]:
def scrape_childcare_data(url):
    response = requests.get(url)

    if response.status_code == 200:
        file = BytesIO(response.content)
        df = pd.read_excel(file)
        # print(df.head())

        # Save the DataFrame as a CSV file
        df.to_csv("../data/external/childcare_facilities.csv", index=False)
        print('Success')
    else:
        raise FileNotFoundError(f"Failed to fetch the data from the URL. Status code: {response.status_code}")

if not path.exists("../data/external/childcare_facilities.csv"):
    scrape_childcare_data("https://shorturl.at/jkQT7")
else:
    print(f'childcare_facilities.csv already exists')

childcare_facilities.csv already exists


In [ ]:
with fiona.open('../data/raw/SA2_2021_AUST_SHP_GDA2020') as fiona_file:
    raw_sa_gdf = gpd.GeoDataFrame.from_features([feature for feature in fiona_file])
    
with fiona.open('../data/raw/catchments') as fiona_file:
    raw_catchments_gdf = gpd.GeoDataFrame.from_features([feature for feature in fiona_file])
    
raw_business_df = pd.read_csv('../data/raw/Businesses.csv', )
raw_income_df = pd.read_csv('../data/raw/Income.csv')
raw_polling_df = pd.read_csv('../data/raw/PollingPlaces2019.csv')
raw_population_df = pd.read_csv('../data/raw/Population.csv')
raw_stops_df = pd.read_csv('../data/raw/Stops.txt')
raw_childcare_df = pd.read_csv('../data/external/childcare_facilities.csv')
raw_poi_gdf = gpd.read_file('../data/external/Points_Of_Interest.json')

In [ ]:
sa2_geometry_gdf = raw_sa_gdf.loc[raw_sa_gdf['GCC_NAME21']=='Greater Sydney',['SA2_CODE21', 'geometry']].rename(columns={'SA2_CODE21':'sa2_code'})
sa2_info_df =  pd.DataFrame(raw_sa_gdf.loc[raw_sa_gdf['GCC_NAME21']=='Greater Sydney',['SA2_CODE21', 'SA2_NAME21', 'AREASQKM21']].rename(columns={'SA2_CODE21':'sa2_code', 'SA2_NAME21':'sa2_name', 'AREASQKM21':'area'}))

sa2_code_list = list(sa2_info_df.sa2_code)

catchments_gdf = raw_catchments_gdf.rename(columns={'USE_ID':'school_id'}).loc[:, ['school_id', 'geometry']]

business_counts_df = raw_business_df.loc[raw_business_df['sa2_code'].astype(str).isin(sa2_code_list), ['industry_code', 'sa2_code', 'total_businesses']]
labels, levels = pd.factorize(business_counts_df['industry_code'])
business_counts_df['industry_code'] = labels
business_counts_df['key'] = business_counts_df['sa2_code'].astype(str) + '_' + business_counts_df['industry_code'].astype(str)

industry_names_df = pd.DataFrame(levels, columns=['industry_code'])
industry_names_df = industry_names_df.merge(raw_business_df.loc[:, ['industry_code', 'industry_name']].drop_duplicates())
industry_names_df['industry_code'] = industry_names_df.index

income_df = raw_income_df.loc[raw_income_df['sa2_code'].astype(str).isin(sa2_code_list)].drop(columns=['sa2_name'])
income_df = income_df.replace({'np': np.nan}).dropna()

polling_gdf = raw_polling_df.loc[~(raw_polling_df['longitude'].isna()) & ~(raw_polling_df['latitude'].isna()), ['polling_place_id', 'longitude', 'latitude']].reset_index(drop=True)
# polling_gdf['geometry'] = gpd.GeoSeries(gpd.points_from_xy(pd.to_numeric(polling_gdf.longitude), pd.to_numeric(polling_gdf.latitude)))
# polling_gdf.drop(columns=['latitude','longitude'], inplace=True)
polling_gdf = gpd.GeoDataFrame(polling_gdf, geometry=gpd.points_from_xy(pd.to_numeric(polling_gdf.longitude), pd.to_numeric(polling_gdf.latitude)))
polling_gdf.drop(columns=['latitude','longitude'], inplace=True)

population_df = raw_population_df.loc[:, ['sa2_code', 'total_people']].rename(columns={'total_people': 'total_population'})
population_df['young_people_population'] = raw_population_df['0-4_people'] + raw_population_df['5-9_people'] + raw_population_df['10-14_people'] + raw_population_df['15-19_people']


stops_gdf = raw_stops_df.loc[raw_stops_df['location_type'].isna(), ['stop_id', 'stop_lon', 'stop_lat']].reset_index(drop=True)
stops_gdf['geometry'] = gpd.GeoSeries(gpd.points_from_xy(stops_gdf['stop_lon'].astype(float), stops_gdf['stop_lat'].astype(float)))
stops_gdf = gpd.GeoDataFrame(stops_gdf, geometry='geometry').drop(columns=['stop_lat', 'stop_lon'])

childcare_geometry = gpd.points_from_xy(raw_childcare_df['LONGITUDE'], raw_childcare_df['LATITUDE'])
childcare_gdf = gpd.GeoDataFrame(raw_childcare_df.ID, geometry=childcare_geometry).rename(columns={'ID': 'childcare_id'})

libraries_gdf = raw_poi_gdf.loc[raw_poi_gdf['poitype']=='Library', ['topoid','geometry']]
libraries_gdf.rename(columns={'topoid': 'library_id'}, inplace=True)

In [ ]:
sa2_geometry_gdf_for_sql = sa2_geometry_gdf.copy()
catchments_gdf_for_sql = catchments_gdf.copy()
polling_gdf_for_sql = polling_gdf.copy()
stops_gdf_for_sql = stops_gdf.copy()
childcare_gdf_for_sql = childcare_gdf.drop_duplicates(subset=['childcare_id']).copy()
libraries_gdf_for_sql = libraries_gdf.copy()

gdfs_to_convert = [
    (sa2_geometry_gdf_for_sql, MultiPolygon), 
    (catchments_gdf_for_sql, MultiPolygon),
    (polling_gdf_for_sql, Point),
    (stops_gdf_for_sql, Point),
    (childcare_gdf_for_sql, Point),
    (libraries_gdf_for_sql, Point)
]

for gdf, type in gdfs_to_convert:
    gdf['geom'] = gdf['geometry'].apply(lambda x: create_wkt_element(x, type, srid=srid))

In [ ]:
# sa2_geometry_gdf.to_file('../data/processed/sa2.geojson', driver='GeoJSON')
# catchments_gdf.to_file('../data/processed/catchments.geojson', driver='GeoJSON')
# polling_gdf.to_file('../data/processed/polling.geojson', driver='GeoJSON')
# stops_gdf.to_file('../data/processed/stops.geojson', driver='GeoJSON')
# childcare_gdf.to_file('../data/processed/childcare_facilities.geojson', driver='GeoJSON')
# libraries_gdf.to_file('../data/processed/libraries.geojson', driver='GeoJSON')

In [ ]:
try:
    conn.close()
    db.dispose()
except Exception as e:
    print('No connection to close.')

db, conn = pgconnect(credentials)
conn.execute(text("set search_path to public"))

Connected successfully.



In [ ]:
with open('../data/sql/schema.sql') as file:
    sql_scheme_string = text(file.read())

conn.execute(sql_scheme_string)

In [ ]:
# conn.rollback()
gdf_to_upload = [
    (sa2_geometry_gdf_for_sql, 'MULTIPOLYGON', 'sa2_geometry'), 
    (catchments_gdf_for_sql, 'MULTIPOLYGON', 'catchments'),
    (polling_gdf_for_sql, 'POINT', 'polling_places'),
    (stops_gdf_for_sql, 'POINT', 'stops'),
    (childcare_gdf_for_sql, 'POINT', 'childcare_facilities'),
    (libraries_gdf_for_sql, 'POINT', 'libraries')
]

for gdf, type, table in gdf_to_upload:
    gdf[[col for col in gdf.columns if col != 'geometry']].to_sql(table, conn, if_exists='append', index=False, dtype={'geom': Geometry(type, srid)})
    

dfs_to_upload = [
    (sa2_info_df, 'sa2_info'),
    (industry_names_df, 'industry_names'),
    (business_counts_df, 'business_counts'),
    (income_df, 'incomes'),
    (population_df, 'populations')
]

for df, table in dfs_to_upload:
    df.to_sql(table, conn, if_exists='append', index=False)

In [ ]:
with open('../data/sql/indexes.sql') as file:
    sql_index_string = text(file.read())

conn.execute(sql_index_string)

In [ ]:
conn.execute(text('set search_path to public'))
with open('../data/sql/views.sql', 'r') as file:
    view_sql_string = text(file.read())

conn.execute(view_sql_string)

In [ ]:
df_for_pca = query(conn, 'select * from metrics')

df_for_pca = df_for_pca.drop(columns=['sa2_code', 'sa2_name', 'geom', 'area', 'young_people_population', 'total_count']).apply(zscore)

cols = list(df_for_pca.columns)

df_for_pca = df_for_pca.to_numpy()

pca = PCA(n_components=1)

pca.fit(df_for_pca)
weights = pca.components_.tolist()

weight_dic = {}
for col_name, weight in zip(cols, weights[0]):
        print(f'{col_name} weight: {weight}')
        weight_dic[f'{col_name}_weight'] = str(weight)
        

polls weight: 0.36725888233772297
stops weight: 0.3542459899107671
schools weight: 0.31043876860537345
retail weight: 0.45075328648827917
health weight: 0.4382001185821598
libraries weight: 0.3236294063075545
childcare_facilities weight: 0.37858235707779997


In [ ]:
conn.execute(text('set search_path to public'))
with open('../data/sql/additional_zscore.sql', 'r') as file:
    scores_sql_string = str(file.read())

scores_with_weight_sql_string = scores_sql_string
for weight_name, weight in weight_dic.items():
    scores_with_weight_sql_string = scores_with_weight_sql_string.replace(weight_name, weight)

scores_with_weight_sql_string = text(scores_with_weight_sql_string)

df = query(conn, scores_with_weight_sql_string)

sa2_geometry_gdf['sa2_code'] = sa2_geometry_gdf['sa2_code'].astype('int64')

df.merge(sa2_geometry_gdf, how='left', on='sa2_code')

,sa2_code,sa2_name,score,geometry
0,117031644,Sydney (North) - Millers Point,4.581597,"MULTIPOLYGON (((151.22538 -33.85525, 151.22525..."
1,117031646,Ultimo,4.011853,"POLYGON ((151.19469 -33.88090, 151.19461 -33.8..."
2,117031336,Surry Hills,3.919028,"POLYGON ((151.20831 -33.88343, 151.20842 -33.8..."
3,117031645,Sydney (South) - Haymarket,3.776552,"POLYGON ((151.19852 -33.87579, 151.19847 -33.8..."
4,119031664,Hurstville - Central,3.606635,"POLYGON ((151.10102 -33.96455, 151.10154 -33.9..."
...,...,...,...,...
354,123031448,The Oaks - Oakdale,-1.190867,"POLYGON ((150.42302 -34.04532, 150.42299 -34.0..."
355,102021049,Jilliby - Yarramalong,-1.191148,"POLYGON ((151.22381 -33.18592, 151.22479 -33.1..."
356,123031446,Douglas Park - Appin,-1.191658,"POLYGON ((150.63446 -34.19147, 150.63454 -34.1..."
357,102011030,Calga - Kulnura,-1.197620,"MULTIPOLYGON (((151.20449 -33.53280, 151.20448..."


      sa2_code                        sa2_name     score   
0    117031644  Sydney (North) - Millers Point  4.581597  \
1    117031646                          Ultimo  4.011853   
2    117031336                     Surry Hills  3.919028   
3    117031645      Sydney (South) - Haymarket  3.776552   
4    119031664            Hurstville - Central  3.606635   
..         ...                             ...       ...   
354  123031448              The Oaks - Oakdale -1.190867   
355  102021049           Jilliby - Yarramalong -1.191148   
356  123031446            Douglas Park - Appin -1.191658   
357  102011030                 Calga - Kulnura -1.197620   
358  115031299       Bilpin - Colo - St Albans -1.205124   

                                              geometry  
0    MULTIPOLYGON (((151.22538 -33.85525, 151.22525...  
1    POLYGON ((151.19469 -33.88090, 151.19461 -33.8...  
2    POLYGON ((151.20831 -33.88343, 151.20842 -33.8...  
3    POLYGON ((151.19852 -33.87579, 151.19847 -33.8

In [ ]:
conn.execute(text('set search_path to public'))
with open('../data/sql/rank_score.sql', 'r') as file:
    rank_scores_sql_string = text(file.read())

query(conn, rank_scores_sql_string)

,sa2_code,sa2_name,score_adv
0,117031644,Sydney (North) - Millers Point,38
1,117031336,Surry Hills,89
2,117031646,Ultimo,92
3,121041417,North Sydney - Lavender Bay,140
4,117031647,Waterloo,163
...,...,...,...
354,102021049,Jilliby - Yarramalong,2250
355,123031446,Douglas Park - Appin,2251
356,102011030,Calga - Kulnura,2259
357,123031448,The Oaks - Oakdale,2261


      sa2_code                        sa2_name  score_adv
0    117031644  Sydney (North) - Millers Point         38
1    117031336                     Surry Hills         89
2    117031646                          Ultimo         92
3    121041417     North Sydney - Lavender Bay        140
4    117031647                        Waterloo        163
..         ...                             ...        ...
354  102021049           Jilliby - Yarramalong       2250
355  123031446            Douglas Park - Appin       2251
356  102011030                 Calga - Kulnura       2259
357  123031448              The Oaks - Oakdale       2261
358  115031299       Bilpin - Colo - St Albans       2278

[359 rows x 3 columns]

In [ ]:
# Read the files
data = pd.read_csv("z_score_test.csv")
data['sa2_code'] = data['sa2_code'].astype(str)
geo_data = gpd.read_file("sa2.geojson")
geo_data['sa2_code'] = geo_data['sa2_code'].astype(str)
data = geo_data.merge(data, on='sa2_code')

In [ ]:
# Calculate and print the descriptive statistics
desc_stats = data.describe()
print(desc_stats)

In [ ]:
# Define matplotlib style
plt.style.use('rose-pine-dawn.mplstyle')

In [ ]:
# Create a histogram showing the overall distribution of scores
plt.hist(data['score'], bins='auto', edgecolor='black', alpha=0.75)
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title('Histogram of Scores')
plt.show()

In [ ]:
# Plot the heatmap of scores
fig, ax = plt.subplots()
data.plot(column='score', cmap='viridis', legend=True, ax=ax)
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
# Read the income file
incomes_df = pd.read_csv('Income.csv')
incomes_df['sa2_code'] = incomes_df['sa2_code'].astype(str)

# Merge the dataframes on the sa2_code column
merged_df = pd.merge(data, incomes_df, on='sa2_code', how='inner')

# Convert the median_income column to float
merged_df['median_income'] = merged_df['median_income'].astype(float)

# Calculate the correlation coefficient
corr_coeff = merged_df['score'].corr(merged_df['median_income'])

print(f'Correlation coefficient between score and median_income: {corr_coeff}')

In [ ]:
# Plot income scatterplot
fig, ax = plt.subplots()
ax.scatter(merged_df['median_income'], merged_df['score'])
ax.set_xlabel('Median Income ($AUD)')
ax.set_ylabel('Score')
ax.set_title('Score vs Median Income')
plt.show()

In [ ]:
# Create an R-tree spatial index
spatial_index = data.sindex

# Find the top-scoring areas
top_scoring_areas = data.nlargest(10, 'score')

# Find adjacent areas to the top-scoring areas
adjacent_areas = set()
for _, top_area in top_scoring_areas.iterrows():
    possible_neighbors = list(spatial_index.intersection(top_area.geometry.bounds))
    for neighbor_index in possible_neighbors:
        # Check if the neighboring area's boundary intersects the top-scoring area's boundary
        if data.loc[neighbor_index].geometry.touches(top_area.geometry):
            adjacent_areas.add(neighbor_index)

# Calculate the average score for the adjacent areas
adjacent_areas_data = data.loc[list(adjacent_areas)]
avg_score_adjacent = adjacent_areas_data['score'].mean()

# Calculate the average score for all areas
avg_score_all = data['score'].mean()

print(f"Average score for adjacent areas: {avg_score_adjacent}")
print(f"Average score for all areas: {avg_score_all}")

# Perform an independent two-sample t-test
t_statistic, p_value = ttest_ind(adjacent_areas_data['score'], data['score'], equal_var=False)

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

# Set a significance level
alpha = 0.05

if p_value < alpha:
    print("The difference in average scores is statistically significant.")
else:
    print("The difference in average scores is not statistically significant.")

In [ ]:
# Initialise a folium map centered at the mean of latitude and longitude
m = folium.Map(location=[data.geometry.centroid.y.mean(),
                         data.geometry.centroid.x.mean()],
               zoom_start=10, control_scale=True, tiles='CartoDBPositron')

# List of metrics and corresponding names for toggles and legend
metrics = [('score', 'Overall Score'),
           ('z_retail', 'Retail Score'),
           ('z_health', 'Health Score'),
           ('z_stops', 'Bus Stop Score'),
           ('z_polls', 'Polling Place Score'),
           ('z_schools', 'School Score'),
           ('z_libraries', 'Library Score'),
           ('z_childcare_facilities', 'Childcare Facilities Score')]

# Generate a choropleth map for each metric
for metric, name in metrics:
    choropleth = folium.Choropleth(
        geo_data=data.__geo_interface__,
        data=data,  # data source
        columns=['sa2_code', metric],
        key_on='feature.properties.sa2_code',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=name,
        name=name,
        show=False
    ).add_to(m)

    # Highlight the highest and lowest scoring areas
    max_score = data[metric].max()
    min_score = data[metric].min()
    folium.GeoJson(
        data[data[metric] == max_score],
        style_function=lambda x: {'fillColor': '#000000', 'color': '#ff0000'},
        name='Highest Score Areas',
    ).add_to(choropleth.geojson)
    folium.GeoJson(
        data[data[metric] == min_score],
        style_function=lambda x: {'fillColor': '#000000', 'color': '#ff0000'},
        name='Lowest Score Areas',
    ).add_to(choropleth.geojson)

    # Add popups
    folium.GeoJson(
        data,
        name='Popups',
        tooltip=GeoJsonTooltip(
            fields=['sa2_name', 'score', 'z_retail', 'z_health', 'z_stops', 'z_polls', 'z_schools', 'z_libraries', 'z_childcare_facilities'],
            aliases=['Area Name', 'Overall Score', 'Retail Score', 'Health Score', 'Bus Stop Score', 'Polling Place Score', 'School Score', 'Library Score', 'Childcare Facilities Score'],
            localize=True
        )
    ).add_to(choropleth.geojson)

# Add layer control to the map
folium.LayerControl(position='bottomleft').add_to(m)

# Save the map
m.save('chlorpleth.html')

In [ ]:
# Load the data
z_score_df = pd.read_csv("z_score_test.csv")
rank_score_df = pd.read_csv("rank_score_test.csv")

# Create index columns
z_score_df['index_z'] = range(1, len(z_score_df) + 1)
rank_score_df['index_adv'] = range(1, len(rank_score_df) + 1)

# Merge the two dataframes based on the index columns
merged_df = pd.merge(z_score_df, rank_score_df, on=['sa2_code', 'sa2_name'])

# Compute Spearman rank correlation
correlation, _ = spearmanr(merged_df['index_z'], merged_df['index_adv'])
print(f"The Spearman rank correlation coefficient is: {correlation}")

# Identify outliers
merged_df['discrepancy'] = abs(merged_df['index_z'] - merged_df['index_adv'])
outliers = merged_df[merged_df['discrepancy'] > 50]

# Create scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['index_z'], merged_df['index_adv'], color='gray', label='Data')
plt.scatter(outliers['index_z'], outliers['index_adv'], color='orange', label='Outliers')

# Annotate outliers
for _, row in outliers.iterrows():
    plt.annotate(row['sa2_name'], (row['index_z'], row['index_adv']))

# Customise the plot
plt.xlabel('Z Score Rank')
plt.ylabel('Rank Score Rank')
plt.title('Comparison of Relative Ranks in Different Scoring Methods')
plt.legend().set_visible(False)

plt.show()

In [ ]:
# Compute deviation from the diagonal line y=x
merged_df['deviation'] = merged_df['index_adv'] - merged_df['index_z']

# Create a deviation-from-baseline plot
plt.figure(figsize=(10, 6))
plt.plot(merged_df['index_z'], merged_df['deviation'], linestyle='-', marker='o', markersize=4)
plt.xlabel('Z Score Rank')
plt.ylabel('Deviation from Monotonic, Linear Relationship')
plt.axhline(y=0, color='black', linestyle='--')
plt.title('Distribution of Deviations From Baseline')

plt.show()

In [ ]:
# df_for_pca = query(conn, 'select * from metrics')

# df_scores = df_for_pca.drop(columns=['sa2_code', 'sa2_name', 'geom', 'area', 'young_people_population', 'total_count']).apply(zscore)

# df_sa2 = df_for_pca[['sa2_code', 'sa2_name']]

# cols = list(df_scores.columns)

# df_scores = df_scores.to_numpy()

# pca = PCA(n_components=2)

# pca.fit(df_scores)
# weights = pca.components_.tolist()

# print([x for x in zip(cols, weights[0])])
# print([x for x in zip(cols, weights[1])])

# # print(pca.transform(df_scores))

# dim_reductuon_df = pd.DataFrame(pca.transform(df_scores), columns=['dim1', 'dim2'])

# dim_reductuon_df.join(df_sa2)

In [ ]:
# conn.close()
# db.dispose()